In [1]:
# 라이브러리 및 구글 시트 연동
import gspread
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pingouin as pg
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from oauth2client.service_account import ServiceAccountCredentials

warnings.filterwarnings('ignore')

scope = 'https://spreadsheets.google.com/feeds'
json = 'kjuexcel-32e0651f6f16.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json, scope)
gc = gspread.authorize(credentials)
sheet_url = 'https://docs.google.com/spreadsheets/d/1TnSqSxkjJE8GPJgZpUCO9tOkpVzQYacKppZzC1H5Pek/edit?usp=sharing'
sdoc = gc.open_by_url(sheet_url)

In [2]:
# 데이터셋 불러오기
worksheet = sdoc.worksheet('Final_Dataset')

# 자살률(종속변수)
suicide = worksheet.get('F4:F79')

# 독립변수
aging = worksheet.get('I4:I79')
divorce = worksheet.get('J4:J79')

one_house = worksheet.get('K4:K79')
disabled = worksheet.get('L4:L79')

heal_ins = worksheet.get('M4:M79')
basic = worksheet.get('N4:N79')

society = worksheet.get('O4:O79')
hospital = worksheet.get('P4:P79')


suicide = [float(suicide[x][0]) for x in range(len(suicide))]
aging = [float(aging[x][0]) for x in range(len(aging))]
divorce = [float(divorce[x][0]) for x in range(len(divorce))]
one_house = [float(one_house[x][0]) for x in range(len(one_house))]
disabled = [float(disabled[x][0]) for x in range(len(disabled))]
heal_ins = [float(heal_ins[x][0]) for x in range(len(heal_ins))]
basic = [float(basic[x][0]) for x in range(len(basic))]
society = [float(society[x][0]) for x in range(len(society))]
hospital = [float(hospital[x][0]) for x in range(len(hospital))]

In [3]:
# 다중공선성 제거를 위하여 Standard 정규화 실시
from sklearn.preprocessing import StandardScaler

ind_var = pd.DataFrame({'aging':aging, 'divorce':divorce, 'one_h':one_house, 'dis':disabled, 
                        'heal':heal_ins, 'basic':basic, 'society':society, 'hospital':hospital})
dep_var = pd.DataFrame({'suicide':suicide})

scaler = StandardScaler()
df_std = scaler.fit_transform(ind_var)
ind_var = pd.DataFrame(df_std)
ind_var.columns=['aging', 'divorce', 'one_h', 'dis', 
                 'heal', 'basic', 'society', 'hospital']



vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(ind_var, i)for i in range(ind_var.shape[1])]
vif["features"] = ind_var.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)

vif

,VIF Factor,features
0,1.598999,one_h
1,2.144053,hospital
2,2.440230,basic
3,4.774540,society
4,5.049741,divorce
5,9.612427,heal
6,13.484273,aging
7,19.502415,dis


In [4]:
from sklearn.decomposition import PCA

ind_var.head()

pca = PCA(n_components=1) # 주성분을 몇개로 할지 결정
x = pd.DataFrame({'aging':ind_var.aging, 'dis':ind_var.dis})
printcipalComponents = pca.fit_transform(x)
# principalDf = pd.DataFrame(data=printcipalComponents, columns = ['aging+dis'])

ind_var = ind_var.drop(['aging', 'dis'], axis='columns')
ind_var = ind_var.assign(aging_dis=printcipalComponents)

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(ind_var, i)for i in range(ind_var.shape[1])]
vif["features"] = ind_var.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)



In [5]:
ind_var1 = sm.add_constant(ind_var, has_constant="add")

lsm = sm.OLS(dep_var, ind_var1)
flsm = lsm.fit()
flsm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                suicide   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     30.84
Date:                Tue, 05 Dec 2023   Prob (F-statistic):           9.37e-19
Time:                        11:50:29   Log-Likelihood:                -198.22
No. Observations:                  76   AIC:                             412.4
Df Residuals:                      68   BIC:                             431.1
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.4079      0.398     66.299      0.000      25.613      27.203
divorce        4.1738      0.816      5.118      0.000       2.546       5.801
one_h          1.7651      0.503      3.506      0.001       0.761       2.770
heal           0.4600      1.230      0.374      0.710      -1.995       2.915
basic         -0.0012      0.609     -0.002      0.998      -1.217       1.214
society       -0.7522      0.870     -0.864      0.390      -2.489       0.984
hospital      -0.7445      0.510     -1.460      0.149      -1.762       0.273
aging_dis      1.8315      0.702      2.609      0.011       0.431       3.233
==============================================================================
Omnibus:                        5.674   Durbin-Watson:                   1.543
Prob(Omnibus):                  0.059   Jarque-Bera (JB):                7.375
Skew:                           0.227   Prob(JB):                       0.0250
Kurtosis:                       4.457   Cond. No.                         7.71
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
ind_var1 = ind_var1.drop(['heal', 'basic', 'society', 'hospital'], axis='columns')

lsm = sm.OLS(dep_var, ind_var1)
flsm = lsm.fit()
flsm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                suicide   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     71.57
Date:                Tue, 05 Dec 2023   Prob (F-statistic):           1.48e-21
Time:                        11:50:29   Log-Likelihood:                -200.02
No. Observations:                  76   AIC:                             408.0
Df Residuals:                      72   BIC:                             417.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.4079      0.396     66.628      0.000      25.618      27.198
divorce        3.9163      0.604      6.488      0.000       2.713       5.120
one_h          1.4846      0.415      3.575      0.001       0.657       2.312
aging_dis      1.3787      0.444      3.106      0.003       0.494       2.264
==============================================================================
Omnibus:                        5.592   Durbin-Watson:                   1.525
Prob(Omnibus):                  0.061   Jarque-Bera (JB):                7.338
Skew:                           0.210   Prob(JB):                       0.0255
Kurtosis:                       4.463   Cond. No.                         2.94
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
loca = worksheet.get('B4:B79')
loca = [loca[x][0] for x in range(len(loca))]

CONST = 26.4079
AGING_DIS = 1.3787
DIVORCE = 3.9163
ONE_H = 1.4846

# CONST = 26.4079
# DIVORCE = 4.1738
# ONE_H = 1.7651
# HEAL = 0.4600
# BASIC = -0.0012
# SOCIETY = -0.7522
# HOSPITAL = -0.7445
# AGING_DIS = 1.8315

save = [[], [], [], []]
def regression(dfs, lfs, loc):
    for i in range(len(dfs['heal'])):
        target = lfs['suicide'][i]
        # output = CONST + dfs['divorce'][i] * DIVORCE
        # + dfs['one_h'][i] * ONE_H
        # +  dfs['heal'][i] * HEAL
        # + dfs['basic'][i] * BASIC
        # + dfs['society'][i] * SOCIETY 
        # + dfs['hospital'][i] * HOSPITAL	
        # + dfs['aging_dis'][i] * AGING_DIS
        output = CONST + dfs['aging_dis'][i] * AGING_DIS +  dfs['divorce'][i] * DIVORCE + dfs['one_h'][i] * ONE_H
        error = target - output
        save[0].append(loc[i])
        save[1].append(target)
        save[2].append(output)
        save[3].append(error)
        
regression(ind_var, dep_var, loca)
result = pd.DataFrame({'Location':save[0], 'Target':save[1], 'Output':save[2], 'Error':save[3]})
result.to_csv('reals.csv')
result

# 변수제거 전 MSE = 1347
# 변수제거 후 MSE = 859

,Location,Target,Output,Error
0,서울 종로구,21.033333,26.312426,-5.279093
1,서울 중구,21.100000,26.150667,-5.050667
2,서울 용산구,22.633333,22.478333,0.155001
3,서울 성동구,23.900000,20.993930,2.906070
4,서울 광진구,19.366667,23.978942,-4.612275
...,...,...,...,...
71,서천군,45.100000,45.182811,-0.082811
72,청양군,41.200000,38.801554,2.398446
73,홍성군,43.766667,31.866615,11.900052
74,예산군,42.600000,37.107122,5.492878


In [8]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(ind_var1, i)for i in range(ind_var1.shape[1])]
vif["features"] = ind_var1.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)

vif

,VIF Factor,features
0,1.000000,const
1,1.097904,one_h
2,2.319245,divorce
3,2.427873,aging_dis
